In [1]:
import os

In [2]:
%pwd

'd:\\Courses\\Udemy\\Complete MLOps Bootcamp\\Projects\\7_End to End Projects\\end-to-end-data-science-project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Courses\\Udemy\\Complete MLOps Bootcamp\\Projects\\7_End to End Projects\\end-to-end-data-science-project'

In [8]:
import pandas as pd

data = pd.read_csv('artifacts/data_ingestion/covtype.csv')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Wilderness_Area2                    581012 non-null 

In [9]:
data.isnull().sum()

Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Wilderness_Area2                      0
Wilderness_Area3                      0
Wilderness_Area4                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0
Soil_Type11                           0


In [10]:
data.shape

(581012, 55)

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [12]:
from src.project.constants import *
from src.project.utils.common import read_yaml, create_directories
from src.project import logger
import zipfile

In [15]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                  params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)   

        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self)-> DataValidationConfig:
        config=self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])

        data_validation_config= DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema)
        
        return data_validation_config

In [29]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self)-> bool:
        try:
            validation_status = True
            errors = []
          
            data = pd.read_csv(self.config.unzip_data_dir)
            
            # Get actual columns and their dtypes
            all_cols = dict(data.dtypes)  # {col_name: dtype}

            # Get schema columns and dtypes
            all_schema = self.config.all_schema  # {col_name: dtype}

            
            for col, dtype in all_cols.items():
                if col not in all_schema:
                    validation_status = False
                    errors.append(f"Column missing in schema: {col}")
                elif str(dtype) != str(all_schema[col]):
                    validation_status = False
                    errors.append(f"Dtype mismatch for '{col}': expected {all_schema[col]}, got {dtype}")

            # Also check if any schema columns are missing in the dataset
            for col in all_schema:
                if col not in all_cols:
                    validation_status = False
                    errors.append(f"Column missing in dataset: {col}")

            # Save validation status
            with open(self.config.STATUS_FILE, 'w') as f:
                if validation_status:
                    f.write(f"Data Validation found no errors!\n")
                if errors:
                    f.write('Data Validation found errors! Check STATUS_FILE for details.\n')
                    f.write("Errors:\n")
                    for err in errors:
                        f.write(f"{err}\n")
            
            return validation_status

        except Exception as e:
            raise e

In [30]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-09-06 13:19:57,892: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-06 13:19:57,896: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-06 13:19:57,910: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-06 13:19:57,913: INFO: common: created directory at: artifacts]
[2025-09-06 13:19:57,915: INFO: common: created directory at: artifacts/data_validation]
